In [ ]:
# note, you need to add parquet to flink installation, follow instruction as per class session
# you must take new Dockerfile on day2 github folder, then build image, then run the cluster

In [1]:
import os
from pyflink.datastream import StreamExecutionEnvironment
from pyflink.table import StreamTableEnvironment

import get_env
env = get_env.get_remote_env()
t_env = StreamTableEnvironment.create(env)
t_env.get_config().set("parallelism.default", "1")

2025-12-02T03:11:02.838274Z main ERROR Reconfiguration failed: No configuration found for '63e31ee' at 'null' in 'null'
2025-12-02T03:11:04.074408Z Thread-3 ERROR Reconfiguration failed: No configuration found for '761badbc' at 'null' in 'null'


In [2]:
conf = t_env.get_config().get_configuration()
conf.set_string("fs.allowed-fallback-filesystems", "hadoop")
conf.set_string("fs.gs.project.id", "flink-demo-470113")
conf.set_string("fs.gs.auth.service.account.json.keyfile", "/etc/gcp/key.json")

In [3]:
# Drop if exists
t_env.execute_sql("DROP TABLE IF EXISTS movies")

ddl_csv = """
CREATE TABLE movies (
    movieId INT,
    title   STRING,
    genres  STRING
) WITH (
    'connector' = 'filesystem',
    'path'      = 'gs://gk2-datalake/bronze/movies/',
    'format'    = 'csv',
    'csv.ignore-parse-errors' = 'true',
    'csv.source.ignore-first-line' = 'true'
)
"""
t_env.execute_sql(ddl_csv)

# 3) Get the table and print schema
movies = t_env.from_path("movies")
print("=== Schema ===")
movies.print_schema()

# 4) Print the data (bounded filesystem source -> will finish)
print("=== Data ===")
#result = movies.execute()
#result.print()

limited = movies.limit(2)
limited.execute().print()


=== Schema ===
(
  `movieId` INT,
  `title` STRING,
  `genres` STRING
)
=== Data ===
+----+-------------+--------------------------------+--------------------------------+
| op |     movieId |                          title |                         genres |
+----+-------------+--------------------------------+--------------------------------+
| +I |      <NULL> |                          title |                         genres |
| +I |           1 |               Toy Story (1995) | Adventure|Animation|Childre... |
+----+-------------+--------------------------------+--------------------------------+
2 rows in set


In [4]:
t_env.execute_sql("DROP TABLE IF EXISTS movies_parquet")

ddl_parquet = """
CREATE TABLE movies_parquet (
    movieId INT,
    title   STRING,
    genres  STRING
) WITH (
    'connector' = 'filesystem',
    'path'      = 'gs://gk2-datalake/silver/movies/',
    'format'    = 'parquet'
)
"""
t_env.execute_sql(ddl_parquet)


In [5]:
t_env.execute_sql("""
INSERT INTO movies_parquet
SELECT movieId, title, genres
FROM movies
WHERE movieId IS NOT NULL
""")


In [6]:
movies_sql = t_env.sql_query("SELECT * FROM movies_parquet LIMIT 5")
movies_sql.execute().print()

+----+-------------+--------------------------------+--------------------------------+
| op |     movieId |                          title |                         genres |
+----+-------------+--------------------------------+--------------------------------+
| +I |           1 |               Toy Story (1995) | Adventure|Animation|Childre... |
| +I |           2 |                 Jumanji (1995) |     Adventure|Children|Fantasy |
| +I |           3 |        Grumpier Old Men (1995) |                 Comedy|Romance |
| +I |           4 |       Waiting to Exhale (1995) |           Comedy|Drama|Romance |
| +I |           5 | Father of the Bride Part II... |                         Comedy |
+----+-------------+--------------------------------+--------------------------------+
5 rows in set
